<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [1]:
# in the beginning, we (install and) import all the needed packages

# !pip install pandas-datareader
# !pip install yfinance
# !pip install pyreadr
# !pip install fix-yahoo-finance==0.1.30


import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import yfinance as yf


#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp

In [2]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")
tickers.head()

0     TER
1     ADM
2    TMUS
3    IDXX
4     LNC
Name: Ticker, dtype: object

## 3 Market Capitalization Data Scraping

In [3]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

#mkt_cap_df_list

Elapsed time: 1.453 seconds.


In [4]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks

df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names

df_mkt_cap

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3128534298.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3128534298.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3128534298.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3128534298.py:6: 

,Date,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,...,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
0,2009-04-01,0.78,0.00,6.24,2.04,1.84,1.07,31.68,4.98,4.81,...,27.33,13.47,9.12,1.17,14.97,6.26,94.12,6.70,1.93,7.62
1,2009-04-02,0.84,0.00,6.60,2.07,1.69,1.11,32.90,5.38,4.89,...,28.56,13.66,8.65,1.23,15.07,6.91,92.64,7.07,2.00,8.08
2,2009-04-03,0.89,0.00,6.45,2.13,1.79,1.10,33.69,5.52,4.96,...,29.27,13.57,8.76,1.27,14.52,7.01,91.16,7.25,2.07,8.12
3,2009-04-06,0.88,0.00,6.32,2.16,1.66,1.09,33.61,5.47,5.00,...,27.93,13.38,8.79,1.21,14.32,6.92,92.24,7.05,1.93,7.96
4,2009-04-07,0.83,0.00,5.95,2.14,1.79,1.06,32.54,5.41,4.86,...,26.43,13.31,8.50,1.17,15.03,6.61,90.90,6.72,1.80,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,2022-11-28,14.11,54.00,187.96,33.31,6.37,16.68,957.44,19.86,53.96,...,65.00,23.04,51.19,4.16,42.15,64.76,277.94,40.81,24.53,12.21
3440,2022-11-29,13.95,54.11,186.35,34.04,6.46,16.63,941.85,19.86,53.78,...,66.26,22.95,51.39,4.32,42.08,64.60,277.49,41.83,24.81,12.20
3441,2022-11-30,14.61,54.70,189.91,35.45,6.61,17.46,983.84,20.23,54.72,...,67.45,23.65,51.86,4.47,44.07,65.12,281.08,42.30,25.06,12.53
3442,2022-12-01,14.43,51.23,190.35,36.56,6.57,18.01,973.24,20.31,54.25,...,67.39,23.66,51.33,4.90,43.59,65.08,286.41,42.57,24.66,12.34


## 4 Getting stock prices

In [6]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 21.494 seconds.


In [7]:
# load the stock prices into list of dataframes - todo function
success_tickers

['TER',
 'ADM',
 'TMUS',
 'IDXX',
 'LNC',
 'WST',
 'AMZN',
 'DOV',
 'ORLY',
 'BSX',
 'USB',
 'ETR',
 'AZO',
 'PVH',
 'BIIB',
 'ETN',
 'PFE',
 'WMB',
 'FITB',
 'CHRW']

In [8]:
stocks_df_list = sp.load_stocks(success_tickers)
stocks_df_list

[            Date       High        Low       Open      Close     Volume  \
 0     2017-12-06  40.139999  39.110001  39.259998  39.959999  1201200.0   
 1     2017-12-07  40.660000  40.130001  40.270000  40.299999  1498300.0   
 2     2017-12-08  40.980000  40.220001  40.779999  40.509998  1779400.0   
 3     2017-12-11  40.919998  40.119999  40.650002  40.459999  1388200.0   
 4     2017-12-12  41.849998  40.610001  41.730000  40.950001  1837700.0   
 ...          ...        ...        ...        ...        ...        ...   
 1253  2022-11-29  90.870003  88.870003  90.139999  89.190002  1099900.0   
 1254  2022-11-30  93.480003  87.860001  89.669998  93.449997  2426400.0   
 1255  2022-12-01  94.610001  90.410004  94.370003  92.290001  1552900.0   
 1256  2022-12-02  92.809998  90.000000  90.110001  92.669998  1177200.0   
 1257  2022-12-05  93.800003  91.660004  92.150002  93.389999  1483173.0   
 
       Adj Close Name  
 0     38.815983  TER  
 1     39.146248  TER  
 2     39.3502

In [ ]:
# continue work here

In [9]:
url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"

querystring = {"symbol":"AMRN","region":"US"}

headers = {
	"X-RapidAPI-Key": "5641d62f08mshe710d13327c02dep161d86jsn29e93040aa12",
	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"prices":[{"date":1670274004,"open":1.1799999475479126,"high":1.2200000286102295,"low":1.159999966621399,"close":1.1799999475479126,"volume":3024116,"adjclose":1.1799999475479126},{"date":1669991400,"open":1.159999966621399,"high":1.2300000190734863,"low":1.159999966621399,"close":1.2200000286102295,"volume":1146500,"adjclose":1.2200000286102295},{"date":1669905000,"open":1.1799999475479126,"high":1.2300000190734863,"low":1.159999966621399,"close":1.190000057220459,"volume":8254500,"adjclose":1.190000057220459},{"date":1669818600,"open":1.149999976158142,"high":1.1699999570846558,"low":1.1200000047683716,"close":1.149999976158142,"volume":1378300,"adjclose":1.149999976158142},{"date":1669732200,"open":1.1399999856948853,"high":1.149999976158142,"low":1.1200000047683716,"close":1.1399999856948853,"volume":2195200,"adjclose":1.1399999856948853},{"date":1669645800,"open":1.149999976158142,"high":1.190000057220459,"low":1.1200000047683716,"close":1.1200000047683716,"volume":1438900,"adjcl

In [10]:
ts = 1670251405
ts = 1669991400
ts = 1669905000
ts = 1638887400
dt = datetime.fromtimestamp(ts)
#calendertime.append(dt.strftime("%m/%d/%Y"))
print(dt)

2021-12-07 14:30:00


In [11]:
start = datetime(2012,5,31) 
end = datetime(2013,1,30)

Apple = yf.Ticker("AAPL") 

#print(Apple.history(start=start, end=end))
print(Apple.history)


<bound method Ticker.history of Ticker object <AAPL>>


In [ ]:
df = yf.download("AMZN MSFT", start="2019-01-01", end="2020-01-01",group_by="ticker") 
print(df) 
print(df.AMZN)

Exception in thread Thread-6 (_run_via_pool):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/requests/models.py", line 971, in json
Exception in thread Thread-5 (_run_via_pool):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/requests/models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/json/__init__.py", line 346, in loads
    return complexjson.loads(self.text, **kwargs)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/json/decoder.py", line 337, in decode
    return _default_decoder

In [ ]:
apple = getQuotes('AAPL')

In [ ]:
from googlefinance import getQuotes
import json
print(json.dumps(getQuotes('AAPL'), indent=2))

In [ ]:
# TODO - download book to market ratio

In [ ]:
# TODO - download market indeces

In [ ]:
# TODO - perform regression

In [ ]:
# TODO - put results into table

In [ ]:
# TODO - comment on findings

In [ ]:
tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [ ]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
https://www.macrotrends.net/assets/php/market_cap.php?t=AAPL

In [ ]:
get.packages('rvest')
library(rvest)

stocks <- pd.Data.Frame(read.csv("tickers_set.csv", sep=","))
colnames(stocks) <- c("Index","Ticker") #maybe pridat adresu
head(stocks,3)

stock_names <- rep(NA,length(stocks$Ticker))
for (i in seq(1:length(stocks$Ticker))){
  stock_names[i] <- as.character(stocks[i,2])
}

long_names <- data.frame(matrix(NA, nrow = length(stocks$Ticker), ncol = 2))
for (i in seq(1:length(stocks$Ticker))){
  long_names[i,1] <- stock_names[i]
  long_names[i,2] <- gsub(paste0("Location: https://www.macrotrends.net/stocks/charts/", stock_names[i],"/"),"",as.character(curlGetHeaders(paste0("https://www.macrotrends.net/stocks/charts/",stock_names[i]))[5]))
  long_names[i,2] <- gsub("/\r\n","",long_names[i,2])
}
names(long_names) <- c("Ticker", "Long_Name")
print(long_names)
write.csv(long_names,"long_names.csv")


1-Gathering the data [Daniel, Parsa] 

2-Cleaning and building a data set (readable one for the regression function) [Daniel, Parsa] 

In [ ]:
from datetime import date
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf

yf.pdr_override() 

today = date.today()
start_date= “2017–01–01”
end_date=”2019–11–30"